In [1]:
from atena.evaluation.metrics import (
    EvalInstance,
    get_dataframe_all_eval_metrics
)
from atena.simulation.dataset import (
    Dataset,
    DatasetMeta,
    DatasetName,
    SchemaName,
)
from atena.simulation.utils import random_action_generator
from atena.simulation.utils import random_action_generator

# Evaluation Example
Here, we generate for each dataset 12 random actions and we give the aggregated (average for non Tree-Blue metrics and corupus-aggregated for Tree Bleu metrics) score of each metric.

## Creating evaluation instances

In [2]:
eval_instances = []

for schema in SchemaName:
    for dataset_name in schema.dataset_names:
        eval_dataset_meta = DatasetMeta(schema, dataset_name=dataset_name)
        eval_instance = EvalInstance(
            eval_dataset_meta,
            actions_lst=[random_action_generator(Dataset(eval_dataset_meta)) for _ in range(12)]
        )
        eval_instances.append(eval_instance)

## Compute metrics' scores
This can take some time. Go and grab some coffee...

In [3]:
get_dataframe_all_eval_metrics(eval_instances)

,Precision,T-BLUE-1,T-BLUE-2,T-BLUE-3,EDA-Sim
0,0.1687,0.142857,0.071429,0.051206,0.246571
